# Advanced Querying Mongo

Importing libraries and setting up connection

In [5]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [6]:
db = client.get_database("prueba")

In [8]:
db.list_collection_names()

['companies']

In [9]:
collection = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [12]:
namematch = list(collection.find({"name":'Babelgum'}))

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [17]:
moretwo =list(collection.find({"number_of_employees" : {"$gt": 5000}}).sort("number_of_employees",-1).limit(20))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [20]:
proj ={"name": 1, "founded_year": 1}

In [21]:
yearlist=list(collection.find({"founded_year": {"$in": [2000,2001,2002,2003,2004,2005]}},proj))

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [30]:
proj2 ={"name": 1, "ipo": 1}

In [32]:
yearlist2 = list(collection.find({"$and": [{"ipo.valuation_amount": {"$gt":100000000}}, {"founded_year":{"$lt":2010}}]},proj2))

In [33]:
yearlist2

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7a

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [34]:
medcomp=list(collection.find({'$and' : [{'number_of_employees' : {'$lt':1000}},
                                {'founded_year' : {'$gt' : 2005}}]}).limit(10).sort('number_of_employees', 1))

### 6. All the companies that don't include the `partners` field.

In [38]:
lista = ["None"]
npcomp = list(collection.find({"partners": {"$nin": lista}}))
len(npcomp )

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [43]:
nullist=list(collection.find({"category_code": None}))

In [44]:
len(nullist)

2751

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [51]:
proj3={"_id":0 , "name": 1, "number_of_employees": 1}
hth=list(collection.find({"number_of_employees": {"$gt":99, "$lt":1001}},proj3))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [59]:
ipop = list(collection.find().sort("ipo.valuation_amount",-1).limit(5))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [61]:
iemp = list(collection.find().sort("number_of_employees",-1).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [98]:
secondsem=list(collection.find({"founded_month" : {"$gt":6}}).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [102]:
twotho = list(collection.find({"$and": [{"acquisition.price_amount":{"$gt":1000000}},
                                        {"founded_year":{"$lt":2000}}]}))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [79]:
proj5={"_id": 0 , "name": 1, "founded_year": 1}
founded=list(collection.find({"founded_year":{"$nin": [None]}},proj5).sort("founded_year",1).limit(50))

In [80]:
founded

[{'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'WeGame', 'founded_year': 1840},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'The Economist Group', 'founded_year': 1843},
 {'name': 'Pearson', 'founded_year': 1844},
 {'name': 'Associated Press', 'founded_year': 1846},
 {'name': 'Beloit College', 'founded_year': 1846},
 {'name': 'Stiefel', 'founded_year': 1847},
 {'name': 'Siemens', 'founded_year': 1847},
 {'name': 'Pfizer', 'founded_year': 1848},
 {'name': 'Nykredit Realkredit', 'founded_year': 1851},
 {'name': 'Corning', 'founded_year': 1851},
 {'name': 'L

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [81]:
firstsev=list(collection.find({"founded_day":{"$lt": 8}}).sort("acquisition.price_amount",-1).limit(10))

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [87]:
category = list(collection.find({"$and": [{"category_code": "web"}, {"number_of_employees":{"$gt":4000}}]}).sort("number_of_employees",1))


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [93]:
tenmill = list(collection.find({"$and": [{"acquisition.price_currency_code": "EUR"}, {"acquisition.price_amount":{"$gt":10000000}}]}))


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [95]:
proj6 = {"_id": 0, "name": 1, "acquisition":1}
firsttrim=list(collection.find({"acquisition.acquired_month": {"$lt": 7}}, proj6).limit(10))

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [106]:
acquired=list(collection.find({"$and": [{"founded_year":{"$gt":1999, "$lt":2011}},{"acquisition.acquired_year":{"$gt":2010}}]}))

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code